In [128]:
!pip uninstall -y pydantic pydantic_core pydantic-settings langchain langchain-core langchain-community
!pip install --upgrade pydantic langchain langchain-core langchain-community
!pip show pydantic
!pip uninstall -y pydantic pydantic_core pydantic-settings

Found existing installation: pydantic 2.11.3
Uninstalling pydantic-2.11.3:
  Successfully uninstalled pydantic-2.11.3
Found existing installation: pydantic_core 2.33.1
Uninstalling pydantic_core-2.33.1:
  Successfully uninstalled pydantic_core-2.33.1
Found existing installation: pydantic-settings 2.9.1
Uninstalling pydantic-settings-2.9.1:
  Successfully uninstalled pydantic-settings-2.9.1
Found existing installation: langchain 0.3.23
Uninstalling langchain-0.3.23:
  Successfully uninstalled langchain-0.3.23
Found existing installation: langchain-core 0.3.54
Uninstalling langchain-core-0.3.54:
  Successfully uninstalled langchain-core-0.3.54
Found existing installation: langchain-community 0.3.21
Uninstalling langchain-community-0.3.21:
  Successfully uninstalled langchain-community-0.3.21
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached langchain-0.3.23-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.54-py3-none-any.whl.metadata (5.9

In [129]:
!pip install -q cassio
!pip install cassandra
!pip install PyPDF2
!pip install langchain_community
!pip install -qU lang
!pip install -U langchain-ollama
!apt install libfaiss-dev

ERROR: Could not find a version that satisfies the requirement cassandra (from versions: none)
ERROR: No matching distribution found for cassandra
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached pydantic_core-2.33.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached pydantic_settings-2.9.1-py3-none-any.whl (44 kB)
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
Using cached pydantic_core-2.33.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libfaiss-dev is already the newest version (1.7.2-5).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [130]:
import cassio
import requests
from langchain_ollama.llms import OllamaLLM
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.llms import Ollama
from PyPDF2 import PdfReader
from langchain.vectorstores.cassandra import Cassandra
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
ASTRA_DB_APPLICATION_TOKEN=""
ASTRA_DB_ID=""
OLLAMA_API_KEY="https://0818-103-132-152-20.ngrok-free.app/"

In [132]:
 pdfreader=PdfReader('/content/Early_prediction_of_lung_cancer_using_the_LQNN_and_CNN_model (2).pdf')

In [133]:
from typing_extensions import Concatenate
raw_text=''
for i,page in enumerate(pdfreader.pages):
  content=page.extract_text()
  if content:
    raw_text+=content

In [134]:
raw_text[:50]

'Early prediction of lung cancer using the LQNN\nand'

In [135]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [136]:
llm_embeddings = OllamaEmbeddings(base_url=OLLAMA_API_KEY, model="deepseek-r1:1.5b")

In [137]:
astra_vector_store=Cassandra(embedding=llm_embeddings,table_name='qa_mini_demo',session=None,keyspace=None)

In [138]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(
    separator="/n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts=text_splitter.split_text(raw_text)

In [139]:
print("Embedding dimension:", len(llm_embeddings.embed_query("test")))
print("Number of texts passed:", len(texts[:50]))

Embedding dimension: 1536
Number of texts passed: 1


In [140]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
embeddings = llm_embeddings.embed_documents(texts[:50])
print("Embedding shape:", len(embeddings), "texts, dimension:", len(embeddings[0]))
astra_vector_store.add_texts(["text"][:50])
print("Inserted %i headlines." %len(texts[:50]))
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Embedding shape: 1 texts, dimension: 1536
Inserted 1 headlines.


In [ ]:
first_quest=True
while True:
  if first_quest:
    query_text=input(" Enter your question(or type 'quit' to exit)").strip()
  else:
      query_text=input("what is your next question")
  if query_text.lower()=="quit":
    break
  if query_text =="":
    continue
  first_quest=False
  llm = OllamaLLM(base_url=OLLAMA_API_KEY, model="deepseek-r1:1.5b")
  answer=astra_vector_index.query(query_text,llm=llm).strip()
  print(answer)
  for doc in astra_vector_store.similarity_search(query_text,k=5):
    print(doc.page_content)

 Enter your question(or type 'quit' to exit)CNN


<think>
Okay, so I'm trying to understand how the CNN + LQNN model works for detecting lung cancer. Let me break this down step by step. First, I know that CNN stands for Convolutional Neural Network, which is good at processing image data because it uses convolution operations to extract features from images. But in this case, they're adding something called LQNN, which seems like a Quadratic Feature Interaction Network. 

Hmm, what's the difference between CNN and LQNN? Well, CNN processes each pixel individually, applying filters to detect patterns like edges or textures. That works pretty well for medical imaging because it can identify various features in images. But maybe CNN isn't enough on its own because lung cancer detection could require more nuanced information than just individual pixels provide.

Then there's LQNN, which I'm not too familiar with. The user provided some equations and an explanation that says LQNN improves feature representation by capturing quadratic inte

Early prediction of lung cancer using the LQNN
and CNN model
Nivetha R
Department of CSE (AI & ML)
SoE,Dayananda Sagar University
Bengaluru, India
nivetha.r-aiml@dsu.edu.in,
niveyamu7284@gmail.com
I. P ROPOSED METHODOLOGY
Learning Quadratic Neural Networks (LQNNs) and Convo-
lutional Neural Networks (CNNs) are used in the structured
approach to accurately detect lung cancer in the proposed
methodology. A potential methodology for predicting lung
cancer using Reduced Inequalities could follow key steps:
A. Dataset Preparation
Medical lung scan images that have been classified as
malignant (cancerous) or non-cancerous comprise the dataset
used in this investigation. The pictures are taken from hospital
scans or publicly accessible medical imaging datasets like
LIDC-IDRI (Lung Image Database Consortium). To guarantee
a balanced distribution of classes for efficient deep-learning
model training, the dataset is organized.
TABLE I
DATASET COMPOSITION FOR LUNG CANCER DETECTION
Category No. of